## Installation

In [1]:
!pip install -qU \
  langchain-core \
  langchain-google-genai \
  langchain-community

## Set gemini api key

In [2]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")



Enter API key for Google Gemini: ········


### import genai model

In [3]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("Send Hi to my friend!"),
]

In [5]:
model.invoke(messages)


AIMessage(content='There are a few ways to translate "Send Hi to my friend!" depending on the nuance you want to convey:\n\n*   **The most common and direct translation:** "Saluta il mio amico/la mia amica da parte mia!" (Use "amico" for a male friend and "amica" for a female friend). This literally means "Greet my friend on my behalf!"\n\n*   **A slightly more informal option:** "Digli/Dille ciao da parte mia!" (Use "digli" for a male friend and "dille" for a female friend). This translates to "Tell him/her hi from me!"\n\n*   **An even more informal and shorter option:** "Ciao al mio amico/alla mia amica!" (Use "amico" for a male friend and "amica" for a female friend). This is like saying "Hi to my friend!" directly.\n\nSo, the best option depends on your relationship with your friend and the context. If you want to be polite, use the first option. If you want to be casual, use the second or third.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'sa

### converting above code to templates

In [6]:
agent_name = "Jarvis"

In [7]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Defining the system prompt (how the AI should act)
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant {agent_name} that helps translate to italian."
    , input_variables=["agent_name"]
)

In [8]:
message = "Hi"

user_prompt = HumanMessagePromptTemplate.from_template("""
You are tasked to translate the message to Italian

-----
{message}
-----

""",
    input_variables=["message"])

In [9]:
print(user_prompt.format(message="Hi"))

content='\nYou are tasked to translate the message to Italian\n\n-----\nHi\n-----\n\n' additional_kwargs={} response_metadata={}


In [11]:
from langchain.prompts import ChatPromptTemplate

first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

In [29]:
print(first_prompt.format(message="Send Hi to my friend!", agent_name="Jarvis"))

System: You are an AI assistant Jarvis that helps translate to italian.
Human: 
You are tasked to translate the message to Italian

-----
Send Hi to my friend!
-----




In [35]:
prompt = first_prompt.invoke({"agent_name": "Jarvis", "message": "Send Hi to my friend!"})



In [36]:
prompt.to_messages()


[SystemMessage(content='You are an AI assistant Jarvis that helps translate to italian.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='\nYou are tasked to translate the message to Italian\n\n-----\nSend Hi to my friend!\n-----\n\n', additional_kwargs={}, response_metadata={})]

In [37]:
response = model.invoke(prompt)


In [38]:
response

AIMessage(content='Ecco la traduzione:\n\n"Saluta il mio amico!"\n\nOr, if you want to be more informal:\n\n"Manda un saluto al mio amico!"', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ceaaa13b-ce48-4560-abda-8921fcad0e44-0', usage_metadata={'input_tokens': 34, 'output_tokens': 34, 'total_tokens': 68, 'input_token_details': {'cache_read': 0}})

In [39]:
print(response.content)

Ecco la traduzione:

"Saluta il mio amico!"

Or, if you want to be more informal:

"Manda un saluto al mio amico!"


In [26]:
chain_one = (
    {
        "message": lambda x: x["message"], 
        "agent_name": lambda x: x["agent_name"]
    }
    | first_prompt
    | model
    | {"response": lambda x: x.content}
)

In [28]:
chain_one.invoke({"agent_name": "Jarvis", "message": "Send Hi to my friend!"})

{'response': 'Ecco la traduzione:\n\n"Saluta il mio amico/la mia amica!"\n\n**Explanation of options:**\n\n*   If you know the friend is male, use "amico".\n*   If you know the friend is female, use "amica".\n*   If you don\'t know the gender, you can use "amico/amica" or slightly rephrase to something like "Digli/Dille ciao da parte mia!" (Tell him/her hello from me!). However, in most cases, using "Saluta il mio amico/la mia amica!" is perfectly fine.'}